# Part 1 – Data Preparation and Preprocessing

### Imports

In [15]:
# Imports
import string
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import cm
import matplotlib.pyplot as plt
import IPython.display 

### Loading dataset

In [16]:
# Load the data using Pandas
df = pd.read_csv("data.csv")
print("The Dataset consists of: {} rows and {} columns".format(len(df), len(df.columns)))

# Visualize the top 5 first rows
df.head()

The Dataset consists of: 318477 rows and 13 columns


C:\Users\Guang\AppData\Local\Temp\ipykernel_9440\737856287.py:2: DtypeWarning: Columns (0,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data.csv")


,Id,Title,Company,Date,Location,Area,Classification,SubClassification,Requirement,FullDescription,LowestSalary,HighestSalary,JobType
0,37404348,Casual Stock Replenisher,Aldi Stores,2018-10-07T00:00:00.000Z,Sydney,North West & Hills District,Retail & Consumer Products,Retail Assistants,Our Casual Stock Replenishers pride themselves...,NaN,0,30,NaN
1,37404337,Casual Stock Replenisher,Aldi Stores,2018-10-07T00:00:00.000Z,Richmond & Hawkesbury,NaN,Retail & Consumer Products,Retail Assistants,Our Casual Stock Replenishers pride themselves...,NaN,0,30,NaN
2,37404356,RETAIL SALES SUPERSTARS and STYLISTS Wanted - ...,LB Creative Pty Ltd,2018-10-07T00:00:00.000Z,Brisbane,CBD & Inner Suburbs,Retail & Consumer Products,Retail Assistants,BRAND NEW FLAGSHIP STORE OPENING - SUNSHINE PLAZA,NaN,0,30,NaN
3,37404330,Team member - Belrose,Anaconda Group Pty Ltd,2018-10-07T00:00:00.000Z,Gosford & Central Coast,NaN,Retail & Consumer Products,Retail Assistants,Bring it on - do you love the great outdoors a...,NaN,0,30,NaN
4,37404308,"Business Banking Contact Centre Specialist, Ni...",Commonwealth Bank - Business & Private Banking,2018-10-07T00:00:00.000Z,Sydney,Ryde & Macquarie Park,Call Centre & Customer Service,Sales - Inbound,"We are seeking highly articulate, enthusiastic...",NaN,0,30,NaN


### Cleaning with dataset

In [17]:
# Clean the Id column
df["Id"] = df["Id"].replace(to_replace = r"&.*", value = "", regex = True)
# Set column "Id" to type int64
df["Id"] = df["Id"].astype('int64')

# Clean the date column
df["Date"] = df["Date"].replace(to_replace = r"T.*", value = "", regex = True)
# Set column "Date" to type datetime64[ns]
df["Date"] = pd.to_datetime(df["Date"])

# Calculate the average salary
df = df.assign(AverageSalary = lambda row : (row.LowestSalary + row.HighestSalary) / 2)

# Data types after data cleaning and conversions
print(df.dtypes)

Id                            int64
Title                        object
Company                      object
Date                 datetime64[ns]
Location                     object
Area                         object
Classification               object
SubClassification            object
Requirement                  object
FullDescription              object
LowestSalary                  int64
HighestSalary                 int64
JobType                      object
AverageSalary               float64
dtype: object


An additional attribute **AverageSalary** has been creating to aid in exploring relationships between different attributes. This was found by calculating the mean between the **LowestSalary** and **HighestSalary**.

### Checking for duplicates

In [18]:
# Finding how many duplicates
duplicate_count = df[df.duplicated()].shape[0]
print("Duplicates found:", duplicate_count)

# Finding how many duplicates there are without "Date" attribute 
no_timestamp = df.drop("Date", axis=1)
duplicate_count_no_timestamp = no_timestamp[no_timestamp.duplicated()].shape[0]
diff = duplicate_count_no_timestamp - duplicate_count
print('Duplicates found without regard to timestamp: ', diff)

Duplicates found: 108
Duplicates found without regard to timestamp:  43


### Removing duplicates

In [19]:
# Dropping duplicates
df = df.drop_duplicates()
duplicate_count = df[df.duplicated()].shape[0]
print("Duplicates found:", duplicate_count)

Duplicates found: 0


### Checking for missing data

In [20]:
null_data = df.isnull().sum()
total_rows = df.shape[0]

print("Attributes with null data: ")
for i in range(len(null_data)):
    if(null_data[i] > 0):
        print("\t{0}: {1:.2f}%".format(df.columns.values[i], (null_data[i] / total_rows) * 100))

Attributes with null data: 
	Company: 3.77%
	Location: 38.08%
	Area: 61.50%
	Classification: 38.08%
	SubClassification: 38.08%
	Requirement: 0.00%
	FullDescription: 5.07%
	JobType: 5.04%


The **Area** attribute has the highest missing data at **61.50%** It is suspected that only job listings with a **Location** based in a larger city use the **Area** attribute to be more specific regarding the location of the job. 

The **Location**, **Classification** and **SubClassification** attribute all share the same amount of missing data of **38.08%**, there is most likely some relationship between these attributes that has caused this, it is unlikely that this is a coincidence.

### Investigating missing "Area" attribute data

In [21]:
# Creating a new dataframe
df_area = df[(df["Area"].isna() == False) & 
             (df["Location"].isna() == False)]

# Printing all unique locations
print(df_area["Location"].unique())

['Sydney' 'Brisbane' 'Melbourne' 'Perth']


We've created a new dataset excluding all entries where the **Area** and **Location** data is missing, then printing all unique **Location**'s. This information backs our suspicion that the **Area** attribute is only used when the **Location** is a large city and more context is needed. Job listings that are missing the **Area** attribute will not be removed from the dataset.

### Investigating missing "Classification", "SubClassification" and "Location" attribute data

In [22]:
# Creating a new dataframe
df_null_data = df[(df["Classification"].isna() == True) & 
                  (df["SubClassification"].isna() == True) & 
                  (df["Location"].isna() == True)]

# Sorting data by date
null_data_dates = df_null_data.sort_values("Date")["Date"].dropna().unique()

# Grabbing the first date
null_data_start = null_data_dates[0]
print("Start of the missing data:", null_data_start)
# Grabbing the last date
null_data_end = null_data_dates[len(null_data_dates)-1]
print("End of the missing data:", null_data_end)

# Checking all job listings between the start and end date
date_range = (df['Date'] > null_data_start) & (df['Date'] <= null_data_end)
temp_df = df.loc[date_range]
null_listings = temp_df[(temp_df['Classification'].isna() == False) |
                        (temp_df['SubClassification'].isna() == False) | 
                        (temp_df['Location'].isna() == False)]
print("Job listings between the above dates that weren't missing all three attributes:", len(null_listings))

Start of the missing data: 2018-12-16 00:00:00
End of the missing data: 2019-03-13 00:00:00
Job listings between the above dates that weren't missing all three attributes: 0


### Checking for missing attribute data outside the date range

In [23]:
df_excluded = df[(df["Date"] < null_data_start) & (df["Date"] > null_data_end)]
print(df_excluded[["Location", "Classification", "SubClassification"]].isnull().sum())

Location             0.0
Classification       0.0
SubClassification    0.0
dtype: float64


The above shows that there is no missing **Classification**, **SubClassification** and **Location** attribute data outside the date range. 

### Examples of Job Listings with missing "Classification", "SubClassification" and "Location" attribute data

In [24]:
# Start date
df_null_data.sort_values('Date').head(1)

,Id,Title,Company,Date,Location,Area,Classification,SubClassification,Requirement,FullDescription,LowestSalary,HighestSalary,JobType,AverageSalary
219272,37939515,Network Engineer,Sonic Human Resourcing Pty Ltd,2018-12-16,NaN,NaN,NaN,NaN,We seek a Network Engineer to provide backfill...,<p>Our client is seeking a Network Engineer to...,150,200,Contract/Temp,175.0


In [25]:
# End date
df_null_data.sort_values('Date').tail(1)

,Id,Title,Company,Date,Location,Area,Classification,SubClassification,Requirement,FullDescription,LowestSalary,HighestSalary,JobType,AverageSalary
318476,38561751,Head of Operations - EOI,Austcorp Executive,2019-03-13,NaN,NaN,NaN,NaN,Expressions of interest for a leadership role,NaN,200,999,NaN,599.5


The above investigation shows that all job listings posted from the **16th of December 2018** to the **13th of March 2019** inclusive, are missing **Classification**, **SubClassification** and **Location** attribute data. 

### Save the cleaned and preprocessed dataset

In [26]:
df.to_csv("preprocessed_data.csv")

# Data Preparation and Preprocessing Summary
The data that was given in .csv format was loaded into a DataFrame using the **read_csv()** method from the **pandas** library.
### Cleaning the data
- Used regex to clean the "Date" & "Id" column before converting the data type of "Date" attribute from **object** to **datetime64[ns]** and "Id" attribute **object** to **int64**
- All other attributes have the correct data types:  
    - **LowestSalary** and **HighestSalary** are **int64**
    - **AverageSalary** are **float64**
    - All other attributes are **objects**
### Duplicated data
- **8607** duplicate job listings were removed form the dataset. We know that these weren't just repeated job postings because the "Date" attribute was include when looking at duplicate entries. 
- There an additional **6724** duplicate job listings without considering the "Date" attribute, this most likely represents job listings that weren't filled and were reposted after some time. These additional duplicates were not removed from the dataset, because these job listings still had useful contextual information.
### Missing data
- The most notable attribute is that **Area** has the highest missing data at **61.50%** It is suspected that only job listings with a **Location** based in the larger cities; **Sydney**, **Brisbane**, **Melbourne** and **Perth** use the **Area** attribute to be more specific regarding the location of the job. This also leads to the early observation that those cities make up at least **38.50%** of the dataset.
- The second observation regarding missing data was that the **Location**, **Classification** and **SubClassification** attribute all shared the same amount of missing data of **38.08%**. After some investigation it was found that all job listings between the **16th of December 2018** to the **13th of March 2019** inclusive, were missing **Classification**, **SubClassification** and **Location** attribute data. Job listings outside the date range were checked for missing data, but nothing of interest was found. Precautions must be taken before drawing conclusions from data related to **Location**, **Classification** and **SubClassification** attributes.
# Hypothesis
- We expect the large majority of job listings to be located in the larger cities; **Sydney**, **Brisbane**, **Melbourne** and **Perth**. With a focus on the eastern coast since that is the more densely populated parts of Australia
- Because the larger cities hold a greater share of the job listings, it should be expected that the average salary would also be higher in these cities compared to remote locations. 
- The most popular job listings expected would be Information & Communication Technology, due to the increase in demand and use of technology. 
- The expected highest average salary would be CEO & General Management, but this could be considered an outlier due to the large gap expected between this classification and the rest. The popularity of Information & Communication Technology would also have a competitive aspect to average salary, driving it up.